- Задание 4: использовать BoostARoota для отбора признаков (настраивать или нет - на ваше усмотрение), переобучить модель и оценить изменение качества.

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mp
import xgboost as xgb
from boostaroota import BoostARoota
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import (roc_auc_score, recall_score, accuracy_score, auc, classification_report, \
                            confusion_matrix, plot_confusion_matrix, precision_recall_curve, \
                            plot_precision_recall_curve, roc_curve, plot_roc_curve)

In [4]:
#!pip install BoostARoota

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

warnings.simplefilter("ignore")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
test_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Kaggle_course/data_HW2/assignment_2_test.csv'

In [9]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Kaggle_course/data_HW2/assignment_2_train.csv'

In [10]:
data = pd.read_csv(train_path)

In [11]:
test = pd.read_csv(test_path)

In [12]:
target = data["isFraud"]
data = data.drop(["isFraud"], axis=1)

numerical_features_train = data.select_dtypes(include=np.number)
categorical_features_train = data.select_dtypes(include=["object"])

In [13]:
numerical_features_test = test.select_dtypes(include=np.number)
categorical_features_test = test.select_dtypes(include=["object"])

In [14]:
# список признаков с категорией object на трейне
obj_list = categorical_features_train.columns.to_list()
obj_list, len(obj_list)

(['ProductCD',
  'card4',
  'card6',
  'P_emaildomain',
  'R_emaildomain',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9'],
 14)

In [18]:
# переведём пропуски в пустое значение и в строку (для теста и для трейна)
for feature in obj_list:
    data[feature] = categorical_features_train.fillna('')
    data[feature] = categorical_features_train[feature].astype('str')
    test[feature] = categorical_features_test.fillna('')
    test[feature] = categorical_features_test[feature].astype('str')

In [17]:
from sklearn.preprocessing import LabelEncoder

In [24]:
def categorical_transformation(df):

  categorical_features = df.select_dtypes(include=["object"])
  
  Product_CD_dict = {
    'W': 1, 
    'H': 2, 
    'C': 3, 
    'S': 4, 
    'R': 5}

  df['ProductCD_ord'] = df.ProductCD.map(Product_CD_dict)

  df['card4'] = df.card4.fillna('unknown')
  Card4_dict = {
    'discover': 1, 
    'mastercard': 2, 
    'visa': 3, 
    'american express': 4, 
    'unknown': 5
}
  df['card4_ord'] = df.card4.map(Card4_dict)

  df['card6'] = df.card6.fillna('unknown')
  Card6_dict = {
    'credit': 1, 
    'debit': 2, 
    'debit or credit': 3, 
    'charge card': 4, 
    'unknown': 5
}
  df['card6_ord'] = df.card6.map(Card6_dict)

  df['P_emaildomain'] = df.P_emaildomain.fillna('unknown')
  df['P_emaildomain_ord'] = LabelEncoder().fit_transform(df.P_emaildomain)

  df['R_emaildomain'] = df.R_emaildomain.fillna('unknown')
  df['R_emaildomain_ord'] = LabelEncoder().fit_transform(df.R_emaildomain)

  M_dict = {
    'F':0,
    'T':1,
    'unknown':-1
}
  M4_dict = {
    'M2':3,
    'M0':4,
    'unknown':6,
    'M1':5
}
  df['M4'] = df.M4.fillna('unknown')
  df['M4_ord'] = df.M4.map(M4_dict)

  M_features = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']
  for feature in M_features: 
    df[feature] = df[feature].fillna('unknown')
    df[feature + '_ord'] = df[feature].map(M_dict)
  
  upd_features = categorical_features.columns.to_list()
  upd_features = [i+'_ord' for i in categorical_features.columns.to_list()]

  return df[upd_features]

In [25]:
df_cat_train = categorical_transformation(data[categorical_features_train.columns.to_list()])

In [26]:
df_cat_test = categorical_transformation(test[categorical_features_test.columns.to_list()])

In [27]:
num_list = numerical_features_train.columns.to_list()

In [28]:
df_cat_train_2 = data[num_list].join(df_cat_train)

In [29]:
df_cat_test_2 = test[num_list].join(df_cat_test)

In [30]:
x_train, x_valid = train_test_split(
    df_cat_train_2, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [32]:
y_lead = test['isFraud']
x_lead = df_cat_test_2

In [33]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

In [35]:
params = {
    "eta": 0.01,
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(y_train),
    "eval_metric": "auc"
}

In [36]:
model_boost = xgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.808336	valid-auc:0.805649
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.896498	valid-auc:0.885305
[200]	train-auc:0.918953	valid-auc:0.90086
[300]	train-auc:0.932777	valid-auc:0.910343
[400]	train-auc:0.941845	valid-auc:0.916268
[500]	train-auc:0.948494	valid-auc:0.920479
[600]	train-auc:0.953345	valid-auc:0.923835
[700]	train-auc:0.957294	valid-auc:0.926607
[800]	train-auc:0.960719	valid-auc:0.929151
[900]	train-auc:0.963897	valid-auc:0.93166
[1000]	train-auc:0.966835	valid-auc:0.933298
[1100]	train-auc:0.969603	valid-auc:0.935077
[1200]	train-auc:0.971923	valid-auc:0.936472
[1300]	train-auc:0.974069	valid-auc:0.937923
[1400]	train-auc:0.975974	valid-auc:0.939088
[1500]	train-auc:0.977672	valid-auc:0.940207
[1600]	train-auc:0.97931	valid-auc:0.941209
[1700]	train-auc:0.980888	valid-auc:0.941989
[1800]	train-auc:0.982376	valid-auc:0.942825
[1900]	train-auc:

In [38]:
%%time
br = BoostARoota(metric='auc', silent=True)
br.fit(x_train, y_train)

CPU times: user 18min 55s, sys: 4.61 s, total: 19min
Wall time: 10min 8s


In [39]:
br.keep_vars_

0          TransactionID
1         TransactionAmt
2                  card1
3                  card2
4                  card3
             ...        
242    R_emaildomain_ord
243               M3_ord
244               M4_ord
245               M5_ord
246               M6_ord
Name: feature, Length: 232, dtype: object

In [40]:
Xtrain_br = br.transform(x_train)
Xvalid_br = br.transform(x_valid)

dtrain_br = xgb.DMatrix(Xtrain_br, label=y_train)
dvalid_br = xgb.DMatrix(Xvalid_br, label=y_valid)

In [41]:
model_br = xgb.train(
    params,
    dtrain_br,
    num_boost_round=5000,
    evals=[(dtrain_br, "train"), (dvalid_br, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.80875	valid-auc:0.807021
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.896376	valid-auc:0.885659
[200]	train-auc:0.918401	valid-auc:0.900595
[300]	train-auc:0.93239	valid-auc:0.910532
[400]	train-auc:0.941848	valid-auc:0.916725
[500]	train-auc:0.948569	valid-auc:0.920705
[600]	train-auc:0.953405	valid-auc:0.924053
[700]	train-auc:0.957479	valid-auc:0.926677
[800]	train-auc:0.960953	valid-auc:0.929347
[900]	train-auc:0.964238	valid-auc:0.931757
[1000]	train-auc:0.967266	valid-auc:0.933538
[1100]	train-auc:0.970133	valid-auc:0.935423
[1200]	train-auc:0.97241	valid-auc:0.936882
[1300]	train-auc:0.974628	valid-auc:0.938543
[1400]	train-auc:0.976552	valid-auc:0.939726
[1500]	train-auc:0.978345	valid-auc:0.940755
[1600]	train-auc:0.979966	valid-auc:0.941791
[1700]	train-auc:0.981465	valid-auc:0.942692
[1800]	train-auc:0.982897	valid-auc:0.943556
[1900]	train-auc:

In [42]:
new_preds = model_br.predict(dvalid_br)

new_score = roc_auc_score(y_valid, new_preds)

print(f"New BoostARoota score = {round(new_score, 5)}")

Base BoostARoota score = 0.95117


In [43]:
old_preds = model_boost.predict(dvalid)

In [44]:
old_score = roc_auc_score(y_valid, old_preds)

In [45]:
print(f"Old BoostARoota score = {round(old_score, 5)}")

Base BoostARoota score = 0.9524


In [46]:
# Пока что модель Boostaroot показала наилучший скор 